Basically here I just wanted to Web-Scrape RightMove. Selfish motive, I am buying a house currently and I wanted to see how much I can expect the value of my house to look in 'X' amount of years. 

So:
1. First Phase - Build a ML model that can predict the value of house based on 

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

In [3]:
num_pages = 10 # Number of pages to look at - change as needed
urls = []
df = pd.DataFrame()

for idx in range(1,num_pages+1):  # Create list of urls corresponding pages we are going to scrape.
    url = "https://www.rightmove.co.uk/house-prices/roath.html?pageNumber=" + str(idx)
    urls.append(url)
    
for url in urls:
    r = requests.get(url)
    # Parsing the HTML
    soup = BeautifulSoup(r.text, 'html.parser')
    content = json.loads(soup.find_all("script")[-4].text.split(" = ")[1].split(";")[0])  # Convert content of page to a dictionary
    content.pop('isAuthenticated', None)
    content.pop('metadata', None)
    content = content['searchResult']['properties'] # Access only the property data

    data = []
    for property in content:
        address = property.get('address')
        propertyType = property.get('propertyType')
        bedrooms = property.get('bedrooms')
        bathrooms = property.get('bathrooms')
        location = property.get('location')
        latitude = location['lat']
        longitude = location['lng']
        transactions = property.get('transactions')
        for transaction in transactions:
            display_price = transaction['displayPrice']
            date_sold = transaction['dateSold']
            tenure = transaction['tenure']
            new_build = transaction['newBuild']
            data.append({
                'address': address,
                'propertyType': propertyType,
                'bedrooms': bedrooms,
                'bathrooms': bathrooms,
                'latitude': latitude,
                'longitude': longitude,
                'display_price': display_price,
                'date_sold': date_sold
                
            })
    new_df = pd.DataFrame(data)
    df = pd.concat([df,new_df])

# Some basic clean up
print(f"Number of Entries 'as read from RightMove: {len(df)}")
df.dropna(axis=0, inplace=True)# drop empty rows with missing data, it's not worth trying to preserve these.
print(f"Number of Entries 'as read from RightMove: {len(df)}")
df.reset_index(inplace=True)
df.drop(columns='index', inplace=True)

Number of Entries 'as read from RightMove: 735
Number of Entries 'as read from RightMove: 570


In [4]:
df

,address,propertyType,bedrooms,bathrooms,latitude,longitude,display_price,date_sold
0,"9, Stallcourt Avenue, Cardiff CF23 5AL",TERRACED,3.0,2.0,51.49517,-3.15421,"£437,000",10 Jan 2025
1,"9, Stallcourt Avenue, Cardiff CF23 5AL",TERRACED,3.0,2.0,51.49517,-3.15421,"£330,000",18 Apr 2019
2,"87, Clifton Street, Cardiff CF24 1LT",TERRACED,3.0,1.0,51.48374,-3.15580,"£240,000",10 Jan 2025
3,"87, Clifton Street, Cardiff CF24 1LT",TERRACED,3.0,1.0,51.48374,-3.15580,"£207,000",28 Oct 2022
4,"87, Clifton Street, Cardiff CF24 1LT",TERRACED,3.0,1.0,51.48374,-3.15580,"£151,000",23 Jul 2015
...,...,...,...,...,...,...,...,...
565,"43, Inverness Place, Cardiff CF24 4RU",TERRACED,2.0,1.0,51.49356,-3.17160,"£217,000",15 Mar 2024
566,"43, Inverness Place, Cardiff CF24 4RU",TERRACED,2.0,1.0,51.49356,-3.17160,"£206,000",30 Aug 2019
567,"43, Inverness Place, Cardiff CF24 4RU",TERRACED,2.0,1.0,51.49356,-3.17160,"£159,950",1 Dec 2006
568,"43, Inverness Place, Cardiff CF24 4RU",TERRACED,2.0,1.0,51.49356,-3.17160,"£118,000",24 Jan 2003
